In [2]:
import open3d as o3d
import numpy as np
from pcd_helper import *

In [3]:
scene = "08"
file = "000000"

# Path to the Velodyne .bin file and .label file
bin_file_path = f'data/sequences/{scene}/velodyne/{file}.bin'
label_file = f"data/sequences/{scene}/labels/{file}.label"
label_output_path = f"out/predicted_labels/{scene}/{file}.npy"

In [4]:
# Load data

# Read the point cloud data
points = read_velodyne_bin(bin_file_path)

# Extract x, y, z coordinates (first three columns)
xyz = points[:, :3]

# Load labels
labels = load_labels(label_file)
true_colors = label2color(labels)

# Predicted labels
predicted_grid_labels = np.load(label_output_path)
predicted_colors = grid_label2color(xyz, predicted_grid_labels)

In [7]:
# Visualize true labels

# Create Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(true_colors)


# # Add intensity as colors
# intensity = points[:, 3]
# intensity_colors = np.zeros((points.shape[0], 3))
# intensity_colors[:, 0] = intensity  # Using intensity as red color channel for visualization

# pcd.colors = o3d.utility.Vector3dVector(intensity_colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], window_name="Velodyne Point Cloud", width=800, height=600)

In [6]:
# Visualize predicted labels

# Create Open3D PointCloud object
pcd_pred = o3d.geometry.PointCloud()
pcd_pred.points = o3d.utility.Vector3dVector(xyz)
pcd_pred.colors = o3d.utility.Vector3dVector(predicted_colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd_pred], window_name="Velodyne Point Cloud", width=800, height=600)